<a href="https://colab.research.google.com/github/Arush0113/EEG_Pipeline-IITR/blob/main/EEG_emotion_recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

import tensorflow as tf

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
df = pd.read_csv('/content/drive/MyDrive/emotions.csv')
df.columns

Index(['# mean_0_a', 'mean_1_a', 'mean_2_a', 'mean_3_a', 'mean_4_a',
       'mean_d_0_a', 'mean_d_1_a', 'mean_d_2_a', 'mean_d_3_a', 'mean_d_4_a',
       ...
       'fft_741_b', 'fft_742_b', 'fft_743_b', 'fft_744_b', 'fft_745_b',
       'fft_746_b', 'fft_747_b', 'fft_748_b', 'fft_749_b', 'label'],
      dtype='object', length=2549)

In [ ]:
plt.figure(figsize=(20,8))
plt.plot(df['fft_0_a'][:200])
plt.plot(df['fft_0_b'][:200], 'orange')

In [6]:
def preprocess_inputs(dataFrame):
  label_mapping = {'NEGATIVE':0, 'NEUTRAL':1, 'POSITIVE':2}
  dataFrame['label'] = dataFrame['label'].replace(label_mapping)
  X = dataFrame.drop('label', axis=1).copy()
  y = dataFrame['label'].copy()
  X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.7, random_state = 123)

  return X_train, X_test, y_train, y_test

In [7]:
X_train, X_test, y_train, y_test = preprocess_inputs(df)

In [8]:
y_train

993     2
1851    2
1253    0
1228    1
1461    1
       ..
1593    2
1147    1
1766    0
1122    1
1346    0
Name: label, Length: 1492, dtype: int64

In [25]:
input = tf.keras.layers.Input(shape = (X_train.shape[1],))
expand_dims = tf.expand_dims(input, axis = 2)
gru = tf.keras.layers.GRU(128, return_sequences=True)(expand_dims)
flatten = tf.keras.layers.Flatten()(gru)
# dense1 = tf.keras.layers.Dense(10)(flatten)
output = tf.keras.layers.Dense(3)(flatten)

model = tf.keras.models.Model(inputs = input, outputs = output)



In [26]:
model.compile(optimizer = tf.keras.optimizers.SGD(learning_rate=1e-6), loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

In [27]:
history = model.fit(X_train, y_train, batch_size = 32, validation_split = 0.2, epochs = 20)

Epoch 1/20
38/38 [==============================] - 5s 101ms/step - loss: 1.8272 - accuracy: 0.3361 - val_loss: 1.1510 - val_accuracy: 0.3746
Epoch 2/20
38/38 [==============================] - 3s 87ms/step - loss: 1.0924 - accuracy: 0.3495 - val_loss: 1.0696 - val_accuracy: 0.4181
Epoch 3/20
38/38 [==============================] - 3s 87ms/step - loss: 1.0523 - accuracy: 0.3898 - val_loss: 1.0507 - val_accuracy: 0.4482
Epoch 4/20
38/38 [==============================] - 3s 87ms/step - loss: 1.0285 - accuracy: 0.4317 - val_loss: 1.0656 - val_accuracy: 0.4716
Epoch 5/20
38/38 [==============================] - 3s 87ms/step - loss: 0.9986 - accuracy: 0.4593 - val_loss: 1.0661 - val_accuracy: 0.5284
Epoch 6/20
38/38 [==============================] - 3s 87ms/step - loss: 0.9524 - accuracy: 0.4853 - val_loss: 1.0106 - val_accuracy: 0.5585
Epoch 7/20
38/38 [==============================] - 3s 87ms/step - loss: 0.8980 - accuracy: 0.5096 - val_loss: 0.9488 - val_accuracy: 0.5686
Epoch 8/20
3